In [1]:
!git clone https://github.com/bushuyeu/LLM-from-scratch.git

Cloning into 'LLM-from-scratch'...
remote: Enumerating objects: 539, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 539 (delta 118), reused 135 (delta 107), pack-reused 384 (from 1)
Receiving objects: 100% (539/539), 51.92 MiB | 41.90 MiB/s, done.
Resolving deltas: 100% (281/281), done.


In [2]:
%cd LLM-from-scratch/assignment1

/content/LLM-from-scratch/assignment1


In [20]:
!git pull

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 1), reused 5 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 3.51 KiB | 1.17 MiB/s, done.
From https://github.com/bushuyeu/LLM-from-scratch
   f4ef073..0368f4a  main       -> origin/main
Updating f4ef073..0368f4a
Fast-forward
 assignment1/ece496b_basics/tokenizer.py | 239 ++++++++++++++++++++++++++++++++
 1 file changed, 239 insertions(+)
 create mode 100644 assignment1/ece496b_basics/tokenizer.py


In [4]:
!git checkout main

Already on 'main'
Your branch is up to date with 'origin/main'.


No need for Conda environment on Colab

In [21]:
!pip install -e .'[test]'

Obtaining file:///content/LLM-from-scratch/assignment1
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for cs336-basics (pyproject.toml) ... done
  Created wheel for cs336-basics: filename=cs336_basics-1.0.6-py3-none-any.whl size=3237 sha256=6c6cdd49e59e2396ccb791f758e7b27459316d1c43c8b91ea233e53b965ea6b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ij6whtyq/wheels/11/33/89/135a2c84cfc3609d8bf16452627bea272b9f0c2a923d4c36c9
Successfully built cs336-basics
  Attempting uninstall: cs336-basics
    Found existing installation: cs336-basics 1.0.6
    Uninstalling cs336-basics-1.0.6:
      Successfully uninstalled cs336-basics-1.0.6


In [6]:
!pytest tests/test_train_bpe.py

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content/LLM-from-scratch/assignment1
configfile: pytest.ini
plugins: jaxtyping-0.3.9, anyio-4.12.1, typeguard-4.5.0, langsmith-0.7.3
collected 3 items                                                              

tests/test_train_bpe.py::test_train_bpe_speed PASSED                     [ 33%]
tests/test_train_bpe.py::test_train_bpe PASSED                           [ 66%]
tests/test_train_bpe.py::test_train_bpe_special_tokens PASSED            [100%]

============================== 3 passed in 0.90s ===============================


In [7]:
%cd /content

# Remove old data folder and start fresh
!rm -rf data
!mkdir -p data
%cd data

# Download TinyStories
!wget -q --show-progress https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
!wget -q --show-progress https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt

# Download OpenWebText sample
!wget -q --show-progress https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
!gunzip owt_train.txt.gz
!wget -q --show-progress https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_valid.txt.gz
!gunzip owt_valid.txt.gz

# Verify all downloads
import os
print("\n=== Download verification ===")
for f in [
    "TinyStoriesV2-GPT4-train.txt",
    "TinyStoriesV2-GPT4-valid.txt",
    "owt_train.txt",
    "owt_valid.txt"
    ]:
    if os.path.exists(f):
        size = os.path.getsize(f)
        print(f"{f}: {size / 1e9:.2f} GB")
    else:
        print(f"{f}: MISSING!")

%cd /content/LLM-from-scratch/assignment1

/content
/content/data
TinyStoriesV2-GPT4- 100%[===================>]   2.07G   350MB/s    in 7.0s    
TinyStoriesV2-GPT4- 100%[===================>]  21.46M  48.8MB/s    in 0.4s    
owt_train.txt.gz    100%[===================>]   4.28G  93.7MB/s    in 45s     
owt_valid.txt.gz    100%[===================>] 106.61M  64.6MB/s    in 1.7s    

=== Download verification ===
TinyStoriesV2-GPT4-train.txt: 2.23 GB
TinyStoriesV2-GPT4-valid.txt: 0.02 GB
owt_train.txt: 11.92 GB
owt_valid.txt: 0.29 GB
/content/LLM-from-scratch/assignment1


In [8]:
# Imports and helper functions
import os
import time
import pickle
import tracemalloc
from ece496b_basics import train_bpe

os.makedirs("outputs", exist_ok=True)

def safe_decode(b):
    """Safely decode bytes to string, falling back to repr if UTF-8 fails."""
    try:
        return b.decode('utf-8')
    except:
        return repr(b)

def analyze_vocab(vocab, name):
    """Print analysis of a vocabulary."""
    longest_token = max(vocab.values(), key=len)
    avg_len = sum(len(v) for v in vocab.values()) / len(vocab)
    
    print(f"\n{name} Vocabulary Analysis:")
    print(f"  Total tokens: {len(vocab)}")
    print(f"  Merged tokens (non-byte): {len([k for k in vocab if k >= 256])}")
    print(f"  Average token length: {avg_len:.2f} bytes")
    print(f"  Longest token: '{safe_decode(longest_token)}' ({len(longest_token)} bytes)")
    
    # Top 5 longest
    print(f"  Top 5 longest tokens:")
    for tid, tbytes in sorted(vocab.items(), key=lambda x: len(x[1]), reverse=True)[:5]:
        print(f"    ID {tid}: '{safe_decode(tbytes)}' ({len(tbytes)} bytes)")

print("Imports and helpers loaded.")

Imports and helpers loaded.


## TinyStories BPE Training (vocab_size=10000)

In [9]:
# Train BPE on TinyStories
tracemalloc.start()
start_time = time.time()

ts_vocab, ts_merges = train_bpe(
    input_path="/content/data/TinyStoriesV2-GPT4-train.txt",
    vocab_size=10000,
    special_tokens=["<|endoftext|>"]
)

elapsed_time = time.time() - start_time
_, peak_mem = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Training time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")
print(f"Peak memory: {peak_mem / 1e9:.2f} GB")
print(f"Number of merges: {len(ts_merges)}")

analyze_vocab(ts_vocab, "TinyStories")

# Save immediately
with open("outputs/ts_vocab_10k.pkl", "wb") as f:
    pickle.dump(ts_vocab, f)
with open("outputs/ts_merges_10k.pkl", "wb") as f:
    pickle.dump(ts_merges, f)
print("\nSaved to outputs/ts_vocab_10k.pkl and outputs/ts_merges_10k.pkl")

Training time: 700.46 seconds (11.67 minutes)
Peak memory: 0.08 GB
Number of merges: 9743

TinyStories Vocabulary Analysis:
  Total tokens: 10000
  Merged tokens (non-byte): 9744
  Average token length: 5.79 bytes
  Longest token: ' accomplishment' (15 bytes)
  Top 5 longest tokens:
    ID 7159: ' accomplishment' (15 bytes)
    ID 9142: ' disappointment' (15 bytes)
    ID 9378: ' responsibility' (15 bytes)
    ID 3227: ' uncomfortable' (14 bytes)
    ID 3514: ' compassionate' (14 bytes)

Saved to outputs/ts_vocab_10k.pkl and outputs/ts_merges_10k.pkl


## OpenWebText BPE Training (vocab_size=32000)

**Problem (train_bpe_expts_owt)**: Train a byte-level BPE tokenizer on OpenWebText with vocab_size=32,000.

Resource requirements: ≤12 hours (no GPUs), ≤100GB RAM

**Note**: This will take several hours. Consider running overnight.

In [11]:
# Check current memory usage
import psutil
print(f"RAM used: {psutil.virtual_memory().percent}%")
print(f"RAM available: {psutil.virtual_memory().available / 1e9:.1f} GB")

RAM used: 3.2%
RAM available: 53.0 GB


In [12]:
# # Train BPE on OpenWebText with vocab_size=32000
# # Takes several hours — skip if you already have the pkl files.
# # Upload owt_vocab_32k.pkl and owt_merges_32k.pkl to outputs/ instead.
#
# tracemalloc.start()
# start_time = time.time()
#
# owt_vocab, owt_merges = train_bpe(
#     input_path="/content/data/owt_train.txt",
#     vocab_size=32000,
#     special_tokens=["<|endoftext|>"]
# )
#
# elapsed_time = time.time() - start_time
# _, peak_mem = tracemalloc.get_traced_memory()
# tracemalloc.stop()
#
# print("="*50)
# print("OpenWebText BPE Training Complete!")
# print("="*50)
# print(f"Training time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes, {elapsed_time/3600:.2f} hours)")
# print(f"Peak memory: {peak_mem / 1e9:.2f} GB")
# print(f"Number of merges: {len(owt_merges)}")
#
# analyze_vocab(owt_vocab, "OpenWebText")
#
# with open("outputs/owt_vocab_32k.pkl", "wb") as f:
#     pickle.dump(owt_vocab, f)
# with open("outputs/owt_merges_32k.pkl", "wb") as f:
#     pickle.dump(owt_merges, f)
# print("\nSaved to outputs/owt_vocab_32k.pkl and outputs/owt_merges_32k.pkl")

# Upload pre-trained OWT vocab/merges instead
import os
from google.colab import files

os.makedirs("outputs", exist_ok=True)
needed = [f for f in ["outputs/owt_vocab_32k.pkl", "outputs/owt_merges_32k.pkl"] if not os.path.exists(f)]
if needed:
    print("Upload owt_vocab_32k.pkl and owt_merges_32k.pkl:")
    uploaded = files.upload()
    for name, data in uploaded.items():
        dest = os.path.join("outputs", name)
        with open(dest, "wb") as f:
            f.write(data)
        print(f"  Saved {dest} ({len(data)} bytes)")
else:
    print("OWT pkl files already present in outputs/")

OWT pkl files already present in outputs/


## Compare TinyStories vs OpenWebText Tokenizers

**Problem (train_bpe_expts_owt) Part (b)**: Compare and contrast the tokenizers trained on TinyStories vs OpenWebText.

In [13]:
# Load vocabularies (in case running from fresh kernel after long OWT training)
import pickle

try:
    ts_vocab
    print("TinyStories vocab already in memory")
except NameError:
    with open("outputs/ts_vocab_10k.pkl", "rb") as f:
        ts_vocab = pickle.load(f)
    print("Loaded TinyStories vocab from disk")

try:
    owt_vocab
    print("OpenWebText vocab already in memory")
except NameError:
    with open("outputs/owt_vocab_32k.pkl", "rb") as f:
        owt_vocab = pickle.load(f)
    print("Loaded OpenWebText vocab from disk")

TinyStories vocab already in memory
Loaded OpenWebText vocab from disk


In [14]:
# Compare token sets (excluding base 256 bytes)
ts_tokens = set(v for k, v in ts_vocab.items() if k >= 256)
owt_tokens = set(v for k, v in owt_vocab.items() if k >= 256)

shared = ts_tokens & owt_tokens
ts_only = ts_tokens - owt_tokens
owt_only = owt_tokens - ts_tokens

print(f"TinyStories merged tokens: {len(ts_tokens)}")
print(f"OpenWebText merged tokens: {len(owt_tokens)}")
print(f"\nShared tokens: {len(shared)}")
print(f"  ({len(shared)/len(ts_tokens)*100:.1f}% of TinyStories tokens are in OWT)")
print(f"  ({len(shared)/len(owt_tokens)*100:.1f}% of OWT tokens are in TinyStories)")
print(f"\nTokens unique to TinyStories: {len(ts_only)}")
print(f"Tokens unique to OpenWebText: {len(owt_only)}")

TinyStories merged tokens: 9744
OpenWebText merged tokens: 31744

Shared tokens: 7063
  (72.5% of TinyStories tokens are in OWT)
  (22.2% of OWT tokens are in TinyStories)

Tokens unique to TinyStories: 2681
Tokens unique to OpenWebText: 24681


In [15]:
# Helper function (in case cell-8 wasn't run)
def safe_decode(b):
    try:
        return b.decode('utf-8')
    except:
        return repr(b)

# Show example tokens unique to each dataset (sorted by length for interesting examples)
print("Top 15 longest tokens UNIQUE to TinyStories (children's stories):")
for t in sorted(ts_only, key=len, reverse=True)[:15]:
    print(f"  '{safe_decode(t)}'")

print(f"\nTop 15 longest tokens UNIQUE to OpenWebText (web text):")
for t in sorted(owt_only, key=len, reverse=True)[:15]:
    print(f"  '{safe_decode(t)}'")

Top 15 longest tokens UNIQUE to TinyStories (children's stories):
  ' congratulated'
  ' granddaughter'
  ' marshmallows'
  ' veterinarian'
  ' strawberries'
  ' imaginations'
  ' caterpillars'
  ' stethoscope'
  ' grandparent'
  ' firefighter'
  ' disagreeing'
  ' screwdriver'
  ' adventurous'
  ' reluctantly'
  ' automobiles'

Top 15 longest tokens UNIQUE to OpenWebText (web text):
  'ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ'
  '----------------------------------------------------------------'
  '————————————————'
  '================================'
  '________________________________'
  '********************************'
  'ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ'
  '................................'
  '--------------------------------'
  '————————'
  ' disproportionately'
  ' telecommunications'
  ' environmentalists'
  ' responsibilities'
  ' counterterrorism'


In [16]:
# Compare statistics
ts_longest = max(ts_vocab.values(), key=len)
owt_longest = max(owt_vocab.values(), key=len)
ts_avg_len = sum(len(v) for v in ts_vocab.values()) / len(ts_vocab)
owt_avg_len = sum(len(v) for v in owt_vocab.values()) / len(owt_vocab)

print("=" * 50)
print("Summary Statistics")
print("=" * 50)
print(f"{'Metric':<30} {'TinyStories':>12} {'OpenWebText':>12}")
print("-" * 50)
print(f"{'Vocab size':<30} {len(ts_vocab):>12} {len(owt_vocab):>12}")
print(f"{'Avg token length (bytes)':<30} {ts_avg_len:>12.2f} {owt_avg_len:>12.2f}")
print(f"{'Longest token (bytes)':<30} {len(ts_longest):>12} {len(owt_longest):>12}")
print("-" * 50)
print(f"TinyStories longest: '{safe_decode(ts_longest)}'")
print(f"OpenWebText longest: '{safe_decode(owt_longest)}'")

Summary Statistics
Metric                          TinyStories  OpenWebText
--------------------------------------------------
Vocab size                            10000        32000
Avg token length (bytes)               5.79         6.34
Longest token (bytes)                    15           64
--------------------------------------------------
TinyStories longest: ' accomplishment'
OpenWebText longest: 'ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ'


## Longest OpenWebText Token (Helper)

Run this cell to report the longest token in the trained OpenWebText vocabulary.

In [17]:
# Find the longest token in OpenWebText vocabulary
import pickle
try:
    owt_vocab
except NameError:
    with open("outputs/owt_vocab_32k.pkl", "rb") as f:
        owt_vocab = pickle.load(f)
# Re-define safe_decode if this is a fresh kernel
def safe_decode(b):
    try:
        return b.decode("utf-8")
    except Exception:
        return repr(b)
longest_id, longest_token = max(owt_vocab.items(), key=lambda x: len(x[1]))
print(f"Longest token ID: {longest_id}")
print(f"Longest token length: {len(longest_token)} bytes")
print(f"Longest token (safe decode): {safe_decode(longest_token)}")
print(f"Longest token (raw bytes): {repr(longest_token)}")


Longest token ID: 25821
Longest token length: 64 bytes
Longest token (safe decode): ÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ
Longest token (raw bytes): b'\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82\xc3\x83\xc3\x82'


## Answers to Assignment Questions

### Part (a): What is the longest token in the OWT vocabulary? Does it make sense?

*Fill in your answer here after running the cells above.*

### Part (b): Compare and contrast TinyStories vs OpenWebText tokenizers.

**Important**: Note that this comparison uses different vocab sizes (TinyStories: 10k, OWT: 32k) as specified by the assignment. When writing your answer, acknowledge this limitation and focus on qualitative differences (token types, domain-specific vocabulary) rather than raw token counts.

*Fill in your answer here. Consider:*
- *The vocab size difference (10k vs 32k) means OWT naturally has more tokens*
- *What % of TinyStories tokens also appear in OWT? What does this suggest?*
- *Types of tokens unique to each (children's vocabulary vs web/technical terms)*
- *Average token length differences and what they indicate about text complexity*

## Optional: Profiling (TinyStories)

In [18]:
# Profile the training to see what takes the most time
import cProfile
import pstats
from io import StringIO

profiler = cProfile.Profile()
profiler.enable()

vocab_prof, merges_prof = train_bpe(
    input_path="/content/data/TinyStoriesV2-GPT4-train.txt",
    vocab_size=10000,
    special_tokens=["<|endoftext|>"]
)

profiler.disable()

s = StringIO()
ps = pstats.Stats(profiler, stream=s).sort_stats('cumulative')
ps.print_stats(20)
print(s.getvalue())

         6507812 function calls (6507706 primitive calls) in 259.846 seconds

   Ordered by: cumulative time
   List reduced from 326 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      259  255.719    0.987  497.458    1.921 {built-in method time.sleep}
      174    0.000    0.000  479.488    2.756 /usr/lib/python3.12/multiprocessing/pool.py:500(_wait_for_updates)
        1    0.000    0.000  240.729  240.729 /usr/lib/python3.12/multiprocessing/pool.py:738(__exit__)
        9    0.000    0.000  240.683   26.743 /usr/lib/python3.12/multiprocessing/util.py:208(__call__)
        1    0.000    0.000  240.683  240.683 /usr/lib/python3.12/multiprocessing/pool.py:654(terminate)
        1    0.000    0.000  240.683  240.683 /usr/lib/python3.12/multiprocessing/pool.py:680(_terminate_pool)
        1    0.025    0.025  240.639  240.639 /usr/lib/python3.12/multiprocessing/pool.py:671(_help_stuff_finish)
        1    0.003    0.003  240.614 

## 2.7(a) Tokenizer Experiments — Compression Ratio

In [22]:
import pickle
import random

from ece496b_basics.tokenizer import Tokenizer

# --- Load saved vocab/merges ---------------------------------------------------
try:
    ts_vocab
    ts_merges
    print("TinyStories vocab/merges already in memory")
except NameError:
    with open("outputs/ts_vocab_10k.pkl", "rb") as f:
        ts_vocab = pickle.load(f)
    with open("outputs/ts_merges_10k.pkl", "rb") as f:
        ts_merges = pickle.load(f)
    print("Loaded TinyStories vocab/merges from disk")

try:
    owt_vocab
    owt_merges
    print("OpenWebText vocab/merges already in memory")
except NameError:
    with open("outputs/owt_vocab_32k.pkl", "rb") as f:
        owt_vocab = pickle.load(f)
    with open("outputs/owt_merges_32k.pkl", "rb") as f:
        owt_merges = pickle.load(f)
    print("Loaded OpenWebText vocab/merges from disk")

# --- Build tokenizers ----------------------------------------------------------
ts_tokenizer = Tokenizer(ts_vocab, ts_merges, special_tokens=["<|endoftext|>"])
owt_tokenizer = Tokenizer(owt_vocab, owt_merges, special_tokens=["<|endoftext|>"])

# --- Sample 10 documents from each training file ------------------------------
def sample_documents(path, n=10, seed=42):
    """Read first ~10 MB, split on <|endoftext|>, and randomly sample n docs."""
    with open(path, "r", encoding="utf-8") as f:
        text = f.read(10_000_000)  # ~10 MB is plenty for sampling
    docs = [d.strip() for d in text.split("<|endoftext|>") if d.strip()]
    random.seed(seed)
    return random.sample(docs, min(n, len(docs)))

ts_docs = sample_documents("/content/data/TinyStoriesV2-GPT4-train.txt")
owt_docs = sample_documents("/content/data/owt_train.txt")

# --- Compute compression ratios -----------------------------------------------
def compression_ratios(docs, tokenizer, label):
    """Compute bytes/token for each document and print a summary table."""
    ratios = []
    print(f"\n{'='*60}")
    print(f"Compression Ratio: {label}")
    print(f"{'='*60}")
    print(f"{'Doc':>4}  {'Bytes':>8}  {'Tokens':>8}  {'Bytes/Token':>12}")
    print(f"{'-'*4}  {'-'*8}  {'-'*8}  {'-'*12}")
    for i, doc in enumerate(docs):
        n_bytes = len(doc.encode("utf-8"))
        token_ids = tokenizer.encode(doc)
        n_tokens = len(token_ids)
        ratio = n_bytes / n_tokens if n_tokens > 0 else float("inf")
        ratios.append(ratio)
        print(f"{i+1:>4}  {n_bytes:>8}  {n_tokens:>8}  {ratio:>12.2f}")
    avg = sum(ratios) / len(ratios)
    print(f"\n  Average compression ratio: {avg:.2f} bytes/token")
    return ratios

ts_ratios = compression_ratios(ts_docs, ts_tokenizer, "TinyStories (10K vocab)")
owt_ratios = compression_ratios(owt_docs, owt_tokenizer, "OpenWebText (32K vocab)")

TinyStories vocab/merges already in memory
Loaded OpenWebText vocab/merges from disk

Compression Ratio: TinyStories (10K vocab)
 Doc     Bytes    Tokens   Bytes/Token
----  --------  --------  ------------
   1       646       150          4.31
   2      1600       393          4.07
   3       839       183          4.58
   4       714       174          4.10
   5       667       159          4.19
   6       734       163          4.50
   7       911       231          3.94
   8      1195       301          3.97
   9      1923       452          4.25
  10       757       164          4.62

  Average compression ratio: 4.25 bytes/token

Compression Ratio: OpenWebText (32K vocab)
 Doc     Bytes    Tokens   Bytes/Token
----  --------  --------  ------------
   1      2292       503          4.56
   2      2293       497          4.61
   3      4518       931          4.85
   4      2985       659          4.53
   5      4656       994          4.68
   6      2050       437          4.69


## 2.7(b) Cross-domain Compression — TinyStories tokenizer on OWT

In [54]:
# Cross-domain: TinyStories tokenizer on OWT documents
owt_cross_ratios = compression_ratios(owt_docs, ts_tokenizer, "OWT docs with TinyStories (10K) tokenizer")


Compression Ratio: OWT docs with TinyStories (10K) tokenizer
 Doc     Bytes    Tokens   Bytes/Token
----  --------  --------  ------------
   1      2292       694          3.30
   2      2293       665          3.45
   3      4518      1884          2.40
   4      2985       902          3.31
   5      4656      1437          3.24
   6      2050       693          2.96
   7      3463      1015          3.41
   8      4302      1231          3.49
   9      5872      1944          3.02
  10      8379      2316          3.62

  Average compression ratio: 3.22 bytes/token


## 2.7 (C)

In [58]:
import os                                                                                                           
os.chdir("/content/LLM-from-scratch/assignment1") 

import time, pickle                                                                                                 
from ece496b_basics.tokenizer import Tokenizer                                                                      
                                                                                                                      
  # Load OWT tokenizer                                                                                              
with open("outputs/owt_vocab_32k.pkl", "rb") as f:
    owt_vocab = pickle.load(f)
with open("outputs/owt_merges_32k.pkl", "rb") as f:
    owt_merges = pickle.load(f)
tokenizer = Tokenizer(owt_vocab, owt_merges, special_tokens=["<|endoftext|>"])

  # Read a chunk of OWT text
with open("/content/data/owt_train.txt", "r", encoding="utf-8") as f:
    sample = f.read(1_000_000)  # ~1 MB

sample_bytes = len(sample.encode("utf-8"))

start = time.time()
tokens = tokenizer.encode(sample)
elapsed = time.time() - start

throughput = sample_bytes / elapsed
print(f"Sample size: {sample_bytes / 1e6:.2f} MB")
print(f"Time: {elapsed:.2f} s")
print(f"Throughput: {throughput / 1e6:.2f} MB/s")

pile_size_gb = 825
pile_time_s = (pile_size_gb * 1e9) / throughput
print(f"\nEstimated time for The Pile (825 GB): {pile_time_s / 3600:.1f} hours")

Sample size: 1.01 MB
Time: 1.50 s
Throughput: 0.67 MB/s

Estimated time for The Pile (825 GB): 342.0 hours


In [ ]:
## 2.7 (d)

In [ ]:
import time
import numpy as np
import pickle
from multiprocessing import Pool, cpu_count
from ece496b_basics.tokenizer import Tokenizer

NUM_WORKERS = cpu_count()
print(f"Using {NUM_WORKERS} workers")

# --- Load tokenizers -----------------------------------------------------------
with open("outputs/ts_vocab_10k.pkl", "rb") as f:
    ts_vocab = pickle.load(f)
with open("outputs/ts_merges_10k.pkl", "rb") as f:
    ts_merges = pickle.load(f)
with open("outputs/owt_vocab_32k.pkl", "rb") as f:
    owt_vocab = pickle.load(f)
with open("outputs/owt_merges_32k.pkl", "rb") as f:
    owt_merges = pickle.load(f)

# --- Multiprocessing helpers ---------------------------------------------------
# Each worker gets its own Tokenizer instance via the initializer,
# avoiding pickling the tokenizer on every chunk.
_worker_tokenizer = None

def init_worker(vocab, merges):
    """Initialize a per-worker tokenizer (called once per process)."""
    global _worker_tokenizer
    _worker_tokenizer = Tokenizer(vocab, merges, special_tokens=["<|endoftext|>"])

def encode_chunk(chunk):
    """Encode a text chunk using the worker's tokenizer."""
    return _worker_tokenizer.encode(chunk)

def parallel_encode(text, vocab, merges, num_workers):
    """Split text on <|endoftext|> boundaries and encode chunks in parallel.

    Splitting on document boundaries ensures we never break a token mid-word.
    """
    docs = text.split("<|endoftext|>")
    # Distribute documents into roughly equal groups
    chunk_size = max(1, len(docs) // num_workers)
    chunks = []
    for i in range(0, len(docs), chunk_size):
        chunks.append("<|endoftext|>".join(docs[i:i + chunk_size]))

    print(f"  Split into {len(chunks)} chunks across {num_workers} workers...", flush=True)

    with Pool(num_workers, initializer=init_worker, initargs=(vocab, merges)) as pool:
        results = pool.map(encode_chunk, chunks)

    # Flatten list of lists into a single token ID sequence
    all_ids = []
    for r in results:
        all_ids.extend(r)
    return all_ids

# --- Encode all datasets -------------------------------------------------------
# Each (name, vocab, merges, input_path, output_path) tuple defines one encoding job.
# Token IDs are stored as uint16: holds 0–65,535, enough for both 10K and 32K vocabs,
# and uses 2 bytes/token (4x smaller than default int64).
datasets = [
    ("TinyStories train", ts_vocab, ts_merges, "/content/data/TinyStoriesV2-GPT4-train.txt", "outputs/ts_train.npy"),
    ("TinyStories valid", ts_vocab, ts_merges, "/content/data/TinyStoriesV2-GPT4-valid.txt", "outputs/ts_valid.npy"),
    ("OWT train", owt_vocab, owt_merges, "/content/data/owt_train.txt", "outputs/owt_train.npy"),
    ("OWT valid", owt_vocab, owt_merges, "/content/data/owt_valid.txt", "outputs/owt_valid.npy"),
]

for name, vocab, merges, path, out_path in datasets:
    print(f"Encoding {name}...", flush=True)
    start = time.time()
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
    ids = parallel_encode(text, vocab, merges, NUM_WORKERS)
    arr = np.array(ids, dtype=np.uint16)
    np.save(out_path, arr)
    elapsed = time.time() - start
    mb_per_sec = len(text.encode("utf-8")) / elapsed / 1e6
    print(f"  {len(arr):,} tokens -> {out_path} ({arr.nbytes / 1e6:.1f} MB) in {elapsed:.1f}s ({mb_per_sec:.1f} MB/s)", flush=True)

print("All done!")

## 4.2 Problem (learning_rate_tuning): Tuning the learning rate (1 pt)

Run the SGD toy example with three learning rates: 1e1, 1e2, and 1e3, for just 10 training iterations. Observe whether loss decays faster, slower, or diverges.

In [ ]:
self, params, lr=1e-3):
        if lr < 0:
            raise ValueError(f"Invalid learning rate: {lr}")
        defaults = {"lr": lr}
        super().__init__(params, defaults)

    def step(self, closure: Optional[Callable] = None):
        loss = None if closure is None else closure()
        for group in self.param_groups:
            lr = group["lr"]
            for p in group["params"]:
                if p.grad is None:
                    continue
                state = self.state[p]
                t = state.get("t", 0)
                grad = p.grad.data
                p.data -= lr / math.sqrt(t + 1) * grad
                state["t"] = t + 1
        return loss


# Run the toy example for lr = 1e1, 1e2, 1e3 (10 steps each)
learning_rates = [1e1, 1e2, 1e3]
results = {}

for lr in learning_rates:
    torch.manual_seed(42)
    weights = torch.nn.Parameter(5 * torch.randn((10, 10)))
    opt = SGD([weights], lr=lr)
    losses = []
    for t in range(10):
        opt.zero_grad()
        loss = (weights**2).mean()
        losses.append(loss.cpu().item())
        loss.backward()
        opt.step()
    results[lr] = losses

# Print results
print(f"{'Step':>4}", end="")
for lr in learning_rates:
    print(f"  {'lr='+str(lr):>12}", end="")
print()
print("-" * 44)

for t in range(10):
    print(f"{t:>4}", end="")
    for lr in learning_rates:
        print(f"  {results[lr][t]:>12.4f}", end="")
    print()

print("\nSummary:")
for lr in learning_rates:
    first, last = results[lr][0], results[lr][-1]
    if last < first * 0.5:
        behavior = "decays well"
    elif last < first:
        behavior = "decays slowly"
    else:
        behavior = "DIVERGES"
    print(f"  lr={lr:.0e}: {first:.4f} -> {last:.4f}  ({behavior})")